In [ ]:
# default_exp examples.blurr_high_level_api

In [ ]:
#all_slow

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Using the high-level Blurr API

> Show all of the high-level `BlurrFor<Task>` classes in action here with the raw data sourced from the huggingface datasets library.

In [ ]:
#export
import torch
from fastai.text.all import *

from datasets import load_dataset, concatenate_datasets
from transformers import *

from blurr.utils import *
from blurr.data.core import *
from blurr.modeling.core import *
from blurr.modeling.question_answering import *

logging.set_verbosity_error()

In [ ]:
#hide_input
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print("Here's what we're running with ...\n")
print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Here's what we're running with ...

Using pytorch 1.7.1
Using fastai 2.4
Using transformers 4.8.1


In [ ]:
#cuda
#hide_input
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Sequence Classification

### Multiclassification (one input)

In [ ]:
#hide
try: del learn; torch.cuda.empty_cache()
except: pass

In [ ]:
raw_datasets = load_dataset('glue', 'cola') 
print(f'{raw_datasets}\n')
print(f'{raw_datasets["train"][0]}\n')
print(f'{raw_datasets["train"].features}\n')

Reusing dataset glue (/home/wgilliam/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

{'idx': 0, 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}

{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['unacceptable', 'acceptable'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}



Capture the indexes for both train and validation sets, use the datasets `concatenate_datasets` to put them into a single dataset, and finally use the `IndexSplitter` method to define our train/validation splits as such:

In [ ]:
n_train, n_valid = raw_datasets['train'].num_rows, raw_datasets['validation'].num_rows
train_idxs, valid_idxs = L(range(n_train)), L(range(n_train, n_train + n_valid))
raw_ds = concatenate_datasets([raw_datasets['train'], raw_datasets['validation']])

In [ ]:
dl_kwargs = {'bs': 4, 'val_bs': 8}
learn_kwargs = { 'metrics': [accuracy] }

# using a List[dict] such as a huggingface dataset
learn = BlearnerForSequenceClassification.from_dictionaries(raw_ds, 'distilroberta-base', 
                                                            text='sentence', label='label',
                                                            dblock_splitter=IndexSplitter(valid_idxs),
                                                            dl_kwargs=dl_kwargs, learner_kwargs=learn_kwargs)

In [ ]:
learn.dls.show_batch(dataloaders=learn.dls, trunc_at=500, max_n=5)

,text,category
0,"Everybody who has ever, worked in any office which contained any typewriter which had ever been used to type any letters which had to be signed by any administrator who ever worked in any department like mine will know what I mean.",1
1,"Ron wanted to wear a tuxedo to the party, but wear a tuxedo to the party Caspar couldn't decide whether to.",0
2,John is prouder of having gone than Sally expected Joan to believe that the man who didn't shave would be.,1
3,"They failed to tell me which problem I'll beat the competition more easily, the sooner I solve.",0


In [ ]:
learn.fit_one_cycle(1, lr_max=2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.541186,0.490749,0.778524,00:52


In [ ]:
learn.show_results(learner=learn, max_n=5)

,text,category,target
0,"Scientists at the South Hanoi Institute of Technology have succeeded in raising one dog with five legs, another with a cow's liver, and a third with no head.",1,1
1,"As a teacher, you have to deal simultaneously with the administration's pressure on you to succeed, and the children's to be a nice guy.",0,0
2,He attributed to a short circuit which was caused by an overloaded transducer the fire which destroyed most of my factory.,1,0
3,Each author whose contribution is written in any language other than English will provide a summary in English.,1,1
4,"Most elections are quickly forgotten, but the election of 2000, everyone will remember for a long time.",1,1


### Multiclassification (two inputs)

In [ ]:
#hide
try: del learn; torch.cuda.empty_cache()
except: pass

In [ ]:
raw_datasets = load_dataset('glue', 'mrpc') 
print(f'{raw_datasets}\n')
print(f'{raw_datasets["train"][0]}\n')
print(f'{raw_datasets["train"].features}\n')

Reusing dataset glue (/home/wgilliam/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

{'idx': 0, 'label': 1, 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}



Capture the indexes for both train and validation sets, use the datasets `concatenate_datasets` to put them into a single dataset, and finally use the `IndexSplitter` method to define our train/validation splits as such:

In [ ]:
n_train, n_valid = raw_datasets['train'].num_rows, raw_datasets['validation'].num_rows
train_idxs, valid_idxs = L(range(n_train)), L(range(n_train, n_train + n_valid))
raw_ds = concatenate_datasets([raw_datasets['train'], raw_datasets['validation']])

In [ ]:
dl_kwargs = {'bs': 4, 'val_bs': 8}
learn_kwargs = { 'metrics': [F1Score(), accuracy] }

# using a List[dict] such as a huggingface dataset
learn = BlearnerForSequenceClassification.from_dictionaries(raw_ds, 'distilroberta-base', 
                                                            text=['sentence1', 'sentence2'], 
                                                            label='label',
                                                            dblock_splitter=IndexSplitter(valid_idxs),
                                                            dl_kwargs=dl_kwargs, learner_kwargs=learn_kwargs)

In [ ]:
learn.dls.show_batch(dataloaders=learn.dls, trunc_at=500, max_n=5)

,text,category
0,"Amrozi accused his brother, whom he called "" the witness "", of deliberately distorting his evidence. Referring to him as only "" the witness "", Amrozi accused his brother of deliberately distorting his evidence.",1
1,"In the evening, he asked for six pizzas and soda, which police delivered. In the evening, he asked for six pepperoni pizzas and two six-packs of soft drinks, which officers delivered.",1
2,"Hormone replacement therapy that combines estrogen and progestin doubles a woman's risk of breast cancer, a British study of more than a million women has found. A major British study has added to the evidence that hormone replacement therapy increases the risk of breast cancer, especially when women receive a combination of estrogen and progestin.",1
3,"The report released Monday simply says, This report does not attempt to address the complexities of this issue. The document stated : "" This report does not attempt to address the complexities of this issue. """,1


In [ ]:
learn.fit_one_cycle(1, lr_max=2e-3)

epoch,train_loss,valid_loss,f1_score,accuracy,time
0,0.549731,0.532476,0.823890,0.718137,00:24


In [ ]:
learn.show_results(learner=learn, max_n=5)

,text,category,target
0,"He said the foodservice pie business doesn 't fit the company's long-term growth strategy. "" The foodservice pie business does not fit our long-term growth strategy.",1,1
1,""" But they never climb out of the pot of beer again. "" It's just that they never climb out of the beer again. """,1,1
2,"The broader Standard & Poor's 500 Index.SPX edged down 9 points, or 0.98 percent, to 921. The technology-laced Nasdaq Composite Index <.IXIC > shed 15 points, or 0.98 percent, to 1,492.",0,0
3,Chiron already has nearly 20 percent acceptances from PowderJect's shareholders. Chiron has acceptances from holders of nearly 20 percent of PowderJect shares.,1,1
4,"Speaking for the first time yesterday, Brigitte's maternal aunt said his family was unaware he had was in prison or that he had remarried. Brigitte's maternal aunt said his family was unaware he had been sent to prison, or that he had remarried in Sydney.",1,1


### Multilabel classification

In [ ]:
#hide
try: del learn; torch.cuda.empty_cache()
except: pass

In [ ]:
raw_datasets = load_dataset('civil_comments')
print(f'{raw_datasets}\n')
print(f'{raw_datasets["train"][0]}\n')
print(f'{raw_datasets["train"].features}\n')

Using custom data configuration default
Reusing dataset civil_comments (/home/wgilliam/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab)


DatasetDict({
    train: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 1804874
    })
    validation: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
    test: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
})

{'identity_attack': 0.0, 'insult': 0.0, 'obscene': 0.0, 'severe_toxicity': 0.0, 'sexual_explicit': 0.0, 'text': "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!", 'threat': 0.0, 'toxicity': 0.0}

{'text': Value(dtype='string', id=None), 'toxicity': Value(dtype='float32', id=None), 'severe_toxicity': Value(dtype='float32', id=None), 'obscene': Value(dtype='float32', id=None), 'thr

In [ ]:
lbl_cols =  ['identity_attack', 'insult', 'obscene', 'toxicity', 'severe_toxicity', 'sexual_explicit', 'threat']

In [ ]:
train_ds = raw_datasets['train'].select(range(10000))
valid_ds = raw_datasets['validation'].select(range(2000))

Capture the indexes for both train and validation sets, use the datasets `concatenate_datasets` to put them into a single dataset, and finally use the `IndexSplitter` method to define our train/validation splits as such:

In [ ]:
n_train, n_valid = len(train_ds), len(valid_ds)
train_idxs, valid_idxs = L(range(n_train)), L(range(n_train, n_train + n_valid))
raw_ds = concatenate_datasets([train_ds, valid_ds])

The labels need to be OHE as ints (the raw data has them as floats)

In [ ]:
def make_ohe(item):
    for k in item.keys():
        if (k in lbl_cols):
            item[k] = int(np.round(item[k]))
    return item

raw_ds = raw_ds.map(make_ohe)

Loading cached processed dataset at /home/wgilliam/.cache/huggingface/datasets/civil_comments/default/0.9.0/e7a3aacd2ab7d135fa958e7209d10b1fa03807d44c486e3c34897aa08ea8ffab/cache-615cf128814440d8.arrow


In [ ]:
dl_kwargs = {'bs': 4, 'val_bs': 8}
learn_kwargs = { 'metrics': [F1ScoreMulti(), accuracy_multi] }

# using a List[dict] such as a huggingface dataset
learn = BlearnerForSequenceClassification.from_dictionaries(raw_ds, 'distilroberta-base', 
                                                            text='text', label=lbl_cols,
                                                            dblock_splitter=IndexSplitter(valid_idxs),
                                                            dl_kwargs=dl_kwargs, learner_kwargs=learn_kwargs)

In [ ]:
learn.dls.show_batch(dataloaders=learn.dls, trunc_at=500, max_n=5)

,text,None
0,"Predatory patrol towing isn't a big subject, and there is no advocacy group that is paying any attention to it, but the City of Portland has completely backed off of enforcing state law where the towing predators are operating on private property, and this is Commissioner Novick's failure. He's in charge of towing.\n\nThe City has allowed Retriever Towing to operate in open violation of ADA for years at their NW Quimby lot, and there is absolutely no provision in city ordinance that takes into ac",
1,"I've went to Pizza Jerk three times and to Red Sauce twice. Pizza Jerk's menu is larger and more complicated, and they changed several menu items after the first two, which is why it got that third visit.\n\nThe photos are assigned after all of my visits (so that we don't tip off owners to a coming review) and also after the first draft of my review is written. I consult with the art director about what to take photos of. In this case, yes, I did say to take more shots of Pizza Jerk.\n\nA few days",
2,"Yes the police officer seems to run toward the truck at first....this sort of reaction sometimes happens when someone panics when a several ton vehicle is coming at them. When you threaten with a b.s. ""we're armed and there'll be no violence so long as we get to do whatever white privilege destruction of Pauite artifacts we want, as long as we can ignore (see the New Yorker) the facts that in private hands land use the feds provide would cost likely three times as much, that the original owners",
3,"Outsider: Here's some more info for you. It's based on the Oregon Department of Employment's publication ""Oregon's Minimum Wage Jobs: Facxts, Figures and Context""\nFacts: 100,000 minimum wage jobs out of 1.9 million about 5%\n Total wages: about 1% of total wages in the State\n \nMy conclusion: the Minimum wage increase will represent a.05% increase in the wages paid in Oregon.\n\nThe article's summary:\n\n""On the whole, small to moderate increases in the minimum wage do n",


In [ ]:
learn.fit_one_cycle(1, lr_max=2e-3)

epoch,train_loss,valid_loss,f1_score,accuracy_multi,time
0,0.038633,0.040438,0.159195,0.987785,01:10


/home/wgilliam/miniconda3/envs/blurr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
learn.show_results(learner=learn, trun_at=500, max_n=5)

/home/wgilliam/miniconda3/envs/blurr/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


,text,None,target
0,"Everyone tries to hack everyone else. I have no doubt Russia would try to hack even canada. However, the US has been doing the same, if we recall Snowden.\n\nEven Merkel's phone conversations were being tapped by the CIA. \n\nThe real purpose of this issue is political. Trump is upset because people are trying to imply that he didn't deserve his victory, that the Russians helped him. It's an ego thing. Good CEOs sometimes have giant egos. I have no problem with that as long as they produce results, I gladly buy shares in their company.\n\nOtoh, Russia did invade Crimea recently, and their missile brought down a commercial airliner and killed lots of innocent people. The world has a right to be annoyed at the Russians.\n\nIf you want to find evidence of Russians hacking, you will find them. But if you want to find China or some guy in a basement somewhere, I have no doubt you can find the same as well. Whether they succeeded or not, that's hard to prove, but there's lots of blackhats",,[]
1,"Thank you for this article.\nErrata: My father, Dr. James Ford Lewis, was minister of the First Unitarian Church in Portland in '58-'60; he wasn't Portland's ""first Unitarian minister."" The church was founded here in 1867 (http://bit.ly/29ClQCp). \nI met Mr. Ellison at Sacramento City College, not at my own university.\nAt PCS, for ""Astoria"" I am coaching the Shoshoni language, not ""Shoshona,"" which does not exist, and there is no such thing as ""Scotch-Canadian patois."" The Scots-Canadian accent is what I spoke of. The Iowa, Arikara, Hawaiian languages will also be heard, along with another 10+ accents of English. \nMy work as OnStar's voice began with years of work at General Magic, where I recorded tens of thousands of prompts (rather than ""a succession of responses"") for a system, not doomed but premature, that supported 2.5 million users at its peak. (http://bit.ly/29Q7P7a). And I am the longest-working pro voice in *speech recognition,* not in general. That would be some bold claim!",,[]
2,"Zuri, I mostly agree with you. Yes, i see that many people face those problems. Where our office / internet cafe is in Waianae is the official Homeless Outreach center for the whole Waianae Coast. Most people that want to move into the area shelters have to pass through our office to process the paperwork. So i see first hand the problems. \n.\nBut that ""minority"" you are referring to, is in fact the majority land owners along the coast, and the people that serve on the neighborhood boards, own businesses and the stores.\n.\nMy point is, at some point this community needs to grab opportunity by the ***** and take control of the community, or someone else eventually will. That someone will be land developers, or drug dealers, or both. You cant be passive and expect to accomplish great things. I hear all of the time that the people of Waianae want better. Well, EXCELLENT! Lets start with the basics and have the people on the boards and land owners kick out the drug dealers and other trash.",,[]
3,"Glad to see, as Canadians we are more and more conscious and aware, of all these abuses from immigration, Trudeau can save travel time and costs if he would have paid attention in the first place, foreign labor and foreign student policies hurting the Canadians (middle-class and families and employees and youth! and disadvantaged disabled, homeless and aboriginals) The bad immigration policy list and abuse, goes on and on and on, it's hard to keep track, write it down people! Now it is high-time for all Canadian politicians to take their heads out of the sand, or wherever they may have them, learn something useful to help Canadians, and put a stop to this abuse, nonsense and madness we have been having with too much immigration, and do their jobs! do what's right. Lot's of attention and resources need to be focused on reducing immigration, fixing our bad policies, limiting foreign labor, etc. etc. it is not even funny! Nothin

## Question Answering

In [ ]:
#hide
try: del learn; torch.cuda.empty_cache()
except: pass

In [ ]:
raw_datasets = load_dataset('squad_v2')
print(f'{raw_datasets}\n')
print(f'{raw_datasets["train"][0]}\n')
print(f'{raw_datasets["train"].features}\n')

Reusing dataset squad_v2 (/home/wgilliam/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/ba48bc29b974701e9ba8d80ac94f3e3df924aba41b764dcf9851debea7c672e4)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

{'answers': {'answer_start': [269], 'text': ['in the late 1990s']}, 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-

In [ ]:
train_ds = raw_datasets['train'].select(range(1000))

In [ ]:
def preprocess_ds(ds, hf_arch, hf_config, hf_tokenizer, hf_model, max_seq_len, 
                  context, question, tok_ans_start, tok_ans_end):
    
    def _preprocess(item):
        tok_kwargs = {}
        if(hf_tokenizer.padding_side == 'right'):
            tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(item[question], item[context]), 
                                                           **tok_kwargs)
        else:
            tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(item[context], item[question]), 
                                                           **tok_kwargs)

        tok_ans = hf_tokenizer.tokenize(str(item['answers']['text'][0]), **tok_kwargs)
        
        start_idx, end_idx = 0,0
        
        if(len(tok_input) < max_seq_len):
            for idx, tok in enumerate(tok_input):
                try:
                    if (tok == tok_ans[0] and tok_input[idx:idx + len(tok_ans)] == tok_ans): 
                        start_idx, end_idx = idx, idx + len(tok_ans)
                        break
                except: pass

        item['tokenized_input'] = tok_input
        item['tokenized_input_len'] = len(tok_input)
        item['tok_answer_start'] = start_idx
        item['tok_answer_end'] = end_idx

        return item
    
    ds = ds.map(_preprocess)
    return ds

In [ ]:
pretrained_model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'

learn = BlearnerForQuestionAnswering.from_dataframe(train_ds, pretrained_model_name,
                                                    preprocess_func=preprocess_ds, max_seq_len=256,
                                                    dblock_splitter=RandomSplitter(), dl_kwargs={ 'bs': 4 })

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [ ]:
learn.dls.show_batch(dataloaders=learn.dls, max_n=2, trunc_at=500)

,text,start/end,answer
0,"what magazine said beyonce was the "" world's most beautiful woman ""? in september 2010, beyonce made her runway modelling debut at tom ford's spring / summer 2011 fashion show. she was named "" world's most beautiful woman "" by people and the "" hottest female singer of all time "" by complex in 2012. in january 2013, gq placed her on its cover, featuring her atop its "" 100 sexiest women of the 21st century "" list. vh1 listed her at number 1 on its 100 sexiest artists list. several wax figures of b","(51, 52)",people
1,"where were items from the clothing line displayed? beyonce and her mother introduced house of dereon, a contemporary women's fashion line, in 2005. the concept is inspired by three generations of women in their family, the name paying tribute to beyonce's grandmother, agnez dereon, a respected seamstress. according to tina, the overall style of the line best reflects her and beyonce's taste and style. beyonce and her mother founded their family's company beyond productions, which provides the li","(129, 139)",in destiny's child's shows and tours


In [ ]:
learn.fit_one_cycle(3, lr_max=1e-3)

epoch,train_loss,valid_loss,time
0,2.288779,1.615837,00:47
1,1.378569,1.015918,00:47
2,0.968127,1.050176,00:47


In [ ]:
learn.show_results(learner=learn, skip_special_tokens=True, max_n=2, trunc_at=500)

,text,start/end,answer,pred start/end,pred answer
0,"who did beyonce part ways with during her hiatus? beyonce announced a hiatus from her music career in january 2010, heeding her mother's advice, "" to live life, to be inspired by things again "". during the break she and her father parted ways as business partners. beyonce's musical break lasted nine months and saw her visit multiple european cities, the great wall of china, the egyptian pyramids, australia, english music festivals and various museums and ballet performances.","(50, 52)",her father,"(50, 52)",her father
1,"beyonce became the first female artist to perform solo in 20 years at which stage? in 2011, documents obtained by wikileaks revealed that beyonce was one of many entertainers who performed for the family of libyan ruler muammar gaddafi. rolling stone reported that the music industry was urging them to return the money they earned for the concerts ; a spokesperson for beyonce later confirmed to the huffington post that she donated the money to the clinton bush haiti fund. later that year she beca","(109, 116)",the 2011 glastonbury festival,"(110, 116)",2011 glastonbury festival


## Cleanup

TO DO

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted index.ipynb.
